# Metric Scores for Football Manager Roles and Positions

In [1]:
import pandas as pd

In [2]:
import glob
import os

In [13]:
directory_path = (os.path.join('/Users/User/Documents/Projects/FootballManagerCode/data/*'))
files_and_dirs = glob.glob(directory_path)

if files_and_dirs:
    latest_file = max(files_and_dirs, key=os.path.getctime)
    print("Latest file:", latest_file)
else:
    print("No files found in the specified directory.")

Latest file: /Users/User/Documents/Projects/FootballManagerCode/data\QPR732023.html


In [14]:
# read html file exported by FM 
rawdata_list = pd.read_html(latest_file, header=0, encoding="utf-8", keep_default_na=False)

In [15]:
# turn into a dataframe
df = rawdata_list[0]

For calculating speed, workrate and set piece metrics, I chose to use a geometric mean over an arithmetic mean, (df['Pac'] + df['Acc']) / 2. 

It offers advantages:

1. Balanced Attributes: The geometric mean is particularly sensitive to values that are significantly lower than others. This means if a player has a very high value in one attribute (e.g., Pac) but a low value in another (e.g., Acc), their overall Spd score won't be inflated. 
2. Reduced Skewness.
3. Non-linear scaling - a combination of two attributes might have a more significant impact than the sum of their individual effects. 

In [29]:
# Calculate speed, workrate and set piece scores
df['Spd'] = (df['Pac'] * df['Acc']) ** 0.5
df['Work'] = (df['Wor'] + df['Sta']) ** 0.5
df['SetPiece'] = (df['Jum'] + df['Bra']) ** 0.5

In [41]:
# Calculate the overall sweeper keeper support score
df['sk_support'] = (
    # Essential attributes
    ((df['Agi'] + df['Ref']) * 5) +
    # Core attributes 
    ((df['1v1'] + df['Ant'] + df['Cmd'] + df['Cnt'] + df['Kic'] + df['Pos']) * 3) +
    # Secondary attributes
    ((df['Acc'] + df['Aer'] + df['Cmp'] + df['Dec'] + df['Fir'] + df['Han'] + df['Pas'] + df['Thr'] + df['Vis']) * 1)
) / 37

df['sk_support'] = df['sk_support'].round(1)

In [42]:
# Calculate the overall wingback support role score (wb)
df['wb_support'] = (
    # Essential attributes
    ((df['Wor'] + df['Acc'] + df['Pac'] + df['Sta']) * 5) +
    # Core attributes 
    ((df['Cro'] + df['Mar'] + df['Dri'] + df['Tck'] + df['OtB'] + df['Tea']) * 3) +
    # Secondary attributes
    ((df['Fir'] + df['Pas'] + df['Tec'] + df['Ant'] + df['Cnt'] + df['Dec'] + df['Pos'] + df['Agi'] + df['Bal']) * 1)
) / (5*4 + 3*6 + 1*9)  # The denominators are the sum of weights for each category

df['wb_support'] = df['wb_support'].round(1)

In [43]:
# Calculate the overall ball playing defender support role score (bpd_support)
df['bpd_support'] = (
    # Key attributes
    ((df['Hea'] + df['Mar'] + df['Pas'] + df['Tck'] + df['Cmp'] + 
      df['Pos'] + df['Jum'] + df['Str']) * 5) +
      
    # Desirable attributes 
    ((df['Fir'] + df['Tec'] + df['Agg'] + df['Ant'] + 
      df['Bra'] + df['Cnt'] + df['Dec'] + df['Vis'] + df['Pac']) * 3)
) / (5*8 + 3*9)  # The denominators are the sum of weights for each category

df['bpd_support'] = df['bpd_support'].round(1)